## Import Statements

In [118]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys
import spacy
import os
import pickle
    
from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'
# modelBeingUsed = "glove-wiki-gigaword-300"
# modelBeingUsed = "glove-wiki-gigaword-100"
# modelBeingUsed = "glove-twitter-50"
# modelBeingUsed = "glove-twitter-100"
modelBeingUsed = "glove-twitter-200"

sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words, dataClean


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

## Downloading binaries and models


In [119]:
print("Should I reload the model?")
tstString = input()
if("no" in tstString.lower() or "n" in tstString.lower()):
    print(" didnt reload model! ")
else:
    print("loading {0}!".format(modelBeingUsed));
    fileName = "{}.pickle".format(modelBeingUsed)
    if(os.path.exists(pathToDatasets+fileName)):
        print("loading via pickle!")
        pickle_in = open(pathToDatasets+fileName, "rb")
        word_vectors = pickle.load(pickle_in);
    else:
        print("Pickle didn't exist, therefore loading model!")
        word_vectors = api.load("{0}".format(modelBeingUsed))
        print("-- Saving to pickle file for next time! --")
        pickle_out = open(pathToDatasets+fileName,"wb")
        pickle.dump(word_vectors, pickle_out)
        pickle_out.close()
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    print("Model Loaded!")

Should I reload the model?

loading glove-twitter-200!
loading via pickle!
Model Loaded!


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Global Variables and Global Objects

In [120]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;

nlp = spacy.load('en')


NUMBER_OF_ALTERNATIVES = 7
TWEET_START = 300
NUM_OF_TWEETS = 100


VERBOSE_PRINTING = True
# VERBOSE_PRINTING = False

# USE_SPACY = False
USE_SPACY = True

COLOR_PRINTING = True
#COLOR_PRINTING = False

PRINT_NEUTRAL = True
# PRINT_NEUTRAL = False

# PRINT_ALL_STRINGS = True
PRINT_ALL_STRINGS = False

SHOW_ALTS = 10

punctuation = r"\"#$%&'+-/;<=>?@[\]*^_`{|}~"
LIST_OF_NEGATIONS = ["not"]

## Class for Sentences


In [121]:
from SentenceClass import *

## Utility Code

In [122]:
def printStrings(sentenceObj):
    
    numberOfPrints = 0
    newStrings = generateHTMLObjectsFromSentence(sentenceObj)
    mainSentiment = sentenceObj.ogSentiment;
    listOfSentencesWithSentiments = []
    bestSentiment = -sys.maxsize - 1
    worstSentiment = sys.maxsize
    
    bestSentimentString = "";
    worstSentimentString = "";
    
    for ind, tSentence in enumerate(newStrings):
            alteredTweet = tSentence.getSentence()
            htmlText = tSentence.getHTML()
            sentimentOfNewString = senty.polarity_scores(alteredTweet)['compound']
            newObj = SentenceWithSentiment(alteredTweet, sentimentOfNewString, htmlText)
            listOfSentencesWithSentiments.append(newObj)
            
            if(sentimentOfNewString >= bestSentiment):
                bestSentiment = sentimentOfNewString
                bestSentimentString = htmlText;
            elif(sentimentOfNewString < worstSentiment):
                worstSentiment = sentimentOfNewString
                worstSentimentString = htmlText;
            
            
            numberOfPrints+=1;
            if(numberOfPrints > SHOW_ALTS or PRINT_ALL_STRINGS == False): continue
            if(sentimentOfNewString == mainSentiment or sentimentOfNewString == 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'black')
            elif(sentimentOfNewString > mainSentiment):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'green')
            elif(sentimentOfNewString < mainSentiment and sentimentOfNewString != 0.0):
                if(PRINT_ALL_STRINGS and numberOfPrints <= SHOW_ALTS): displayText("{0}: {1}".format(htmlText,sentimentOfNewString),'red')
                
    if(numberOfPrints > SHOW_ALTS): print("--- More options (total: {0}) possible, but not printed ---".format(numberOfPrints));
    displayText("Actual Sentence: {0} :{1}".format(sentenceObj.ogSentence, sentenceObj.ogSentiment))
    if (worstSentimentString != ""): displayText("Worst Sentence: {0} : {1}".format(worstSentimentString, worstSentiment), color='red')
    if (bestSentimentString != ""): displayText("Best Sentence: {0} : {1}".format(bestSentimentString, bestSentiment), color='green') 
    
    
    sentenceObj.addFinalSentences(listOfSentencesWithSentiments)
    return listOfSentencesWithSentiments, sentenceObj;

def cstr(s, color='black', italics=False):
    if(COLOR_PRINTING):
        if(italics):
            return cstr("<i>{0}</i>".format(s), color);
        return "<text style=color:{}>{}</text>".format(color, s)
    else:
        return "{}".format(s)

def displayText(text, color='black'):
    if(COLOR_PRINTING):
        display(HTML(cstr(text, color)));
        return
    print("{}".format(text));
    
    
def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    if(USE_SPACY == False):
        tags = nltk.pos_tag(tweet)
        return tags;    
    tweet = ' '.join(tweet)
    doc = nlp(tweet)
    tags = [(token.text, token.tag_) for token in doc] # since the format expected is [text,tag]
    return tags

def getAntonymsAndSynonymsOfWords(word):
    if(word not in vocabulary):
        return [], []
    setOfAntonyms = set()
    setOfSynonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            setOfSynonyms.add(l.name().lower())
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name().lower())
    if(len(setOfAntonyms) == 0):
        if(VERBOSE_PRINTING): print("No antonyms found for word {0}".format(word))
    if(len(setOfSynonyms) == 0):
        if(VERBOSE_PRINTING): print("No synonyms found for word {0}".format(word))
            
    return list(setOfAntonyms), list(setOfSynonyms)


def returnReplacementsForWord(word):
    
    if(word not in vocabulary):
        print(" --- {0} not in vocabulary ---".format(word))
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    
    if(possibleReplacements == []):
        print(" --- No replacements for word {0} ---".format(word))
    antonyms,synonyms = getAntonymsAndSynonymsOfWords(word)
    
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        if(VERBOSE_PRINTING): print("Some antonyms for word {0} are {1}".format(word, antonyms[:3]))
    if(synonyms != []):
        possibleReplacements.extend(synonyms)
        if(VERBOSE_PRINTING): print("Some synonyms for word {0} are {1}".format(word, synonyms[:3]))
    return possibleReplacements
    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords == []):
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTags(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    
    for ind,word in enumerate(alternativeWords):
        if("_" in word): continue
        tempTokens[indexOfToken] = word
        posTags = getPOSTags(tempTokens)
        newTag = (posTags[indexOfToken])[1]
        if(str(newTag) == str(mainTag)):
#             if(VERBOSE_PRINTING): print("Word {0}[{1}] matched with {2}[{3}]".format(mainWord, mainTag, word,newTag))
            validWords.append(word)
    if(validWords == [] and VERBOSE_PRINTING):
        print("No POS words found for word {} with tag {}".format(mainWord, mainTag));
    return validWords
    

def removeNegationsFromSentence(sentenceObj):
    finalSentenceObjects = sentenceObj.getFinalSentences();
    newObjs = []
    counterOfRemovals = 0
    for obj in finalSentenceObjects:
        thisSentence = cleanAndTokenizeText(obj.getSentence());
        for word in LIST_OF_NEGATIONS:
            if(word in thisSentence):
                notIndex = thisSentence.index(word)
                if(notIndex >= len(thisSentence) - 1): continue
#                 if(senty.polarity_scores(thisSentence[notIndex+1])['compound'] != 0):
                newSentence = thisSentence[:notIndex] + thisSentence[notIndex+1:] # indexing works as [)
                newSentence = ' '.join(newSentence)
                newSentiment = senty.polarity_scores(newSentence)['compound']
                newObjs.append(SentenceWithSentiment(newSentence, newSentiment))
                sentenceObj.addAlternativesByIndex(notIndex,["IGNORE_FLAG"])
                counterOfRemovals+=1
    if(VERBOSE_PRINTING): print("number of negations removed: {0}".format(counterOfRemovals))
    sentenceObj.addFinalSentences(newObjs)
    return sentenceObj


def generateHTMLObjectsFromSentence(sentenceObj):
    
    allSentences = sentenceObj.getFinalSentences()
    indexToAlts = sentenceObj.indexToSetOfWords;
    indexToChange = list(indexToAlts.keys());
    
    listOfSentenceObjs = []
    for t_sentenceObj in allSentences:
        sentence = t_sentenceObj.getSentence()
        copySentence = cleanAndTokenizeText(sentence)
        for index in indexToChange:
            if(index >= len(copySentence)): continue
            if("IGNORE_FLAG" in indexToAlts[index] and copySentence[index] in LIST_OF_NEGATIONS): continue
            copySentence[index] = cstr("[{0}]".format(copySentence[index]), "blue", italics=True);
        listOfSentenceObjs.append(SentenceWithHTML(sentence, ' '.join(copySentence)));
    
    return listOfSentenceObjs

    
    

## Sentence Chunking and Appending

In [123]:
def helper_combine(mainList, myList):
    '''
    helper function for CombineSentenceChunks
    '''
    newList = []
    for val in myList:
        for mainVal in mainList:
#             if(VERBOSE_PRINTING): print("Combining {0} with {1}".format(' '.join(val), ' '.join(mainVal)));
            newList.append(val + mainVal);
    return newList;

def combineSentenceChunks(wholeSentence, dictOfChunks):
    '''
        Uses the helper_combine function to generate all possible combinatios and permuations of the chunks
        and any alternatives.
        
        The logic is to use the end of the sentence, apply each possible chunk from the previous key's chunks
        to every possible chunk of this key's.
        
        The helper function is used to allow us to reuse the list of alreadyGeneratedChunks and constantly
        append to them.
        
        To understand the logic better, take a look at this gist:
        https://gist.github.com/sunnyMiglani/cf85407a9e6928237b1436cc2bc95fa4
        
    '''
    reversedKeys = sorted(dictOfChunks.keys(), reverse=True)
    completeSentences = [];
    mainArr = dictOfChunks[reversedKeys[0]]
    for ind in range(1, len(reversedKeys)):
        key = reversedKeys[ind];
        mainArr = helper_combine(mainArr, dictOfChunks[key]);
        
    return mainArr;
        
def generateSentenceChunks(wholeSentence, keyToChange, nextKey, listOfMyAlternatives):
    '''
        Generates sentence chunks by iterating through the list of alternatives
        Chunking the sentence to start from current key to next key.
        This means that the sentence always goes from key 'x' to key 'y'
        
        Example:
        "I really <hate> hot chocolate, but I <prefer> hot coffee"
        Calling generateSentenceChunks will create an example sentence:
            - "<altWordForHate> hot chocolate , but I "
        
        Remember to append the first stretch of the string to the first key's chunk for proper use!
    '''
    
    newList = list(listOfMyAlternatives)
    newList.append(wholeSentence[keyToChange]);
    generatedSentences = []
    for myAlt in newList:
        newSentence = wholeSentence[:]
        newSentence[keyToChange] = myAlt
#         if(VERBOSE_PRINTING): print("Generated : {}".format(newSentence[keyToChange:nextKey]))
        generatedSentences.append(newSentence[keyToChange:nextKey]);
        
    return generatedSentences
    
def returnCombinationsOfStrings(sentenceObj):
    
    indexToWordDict = sentenceObj.indexToSetOfWords;
    originalSentence = sentenceObj.ogSentence;
    tokenizedSentence = cleanAndTokenizeText(originalSentence)
    reversedKeys = sorted(indexToWordDict.keys(), reverse=True)
    dictAlternatives  = {}

    sortedKeys = sorted(indexToWordDict.keys())
    sentenceChunks = {}
    htmlChunks = {}
    
    for ind in range(0,len(sortedKeys)):
        key = sortedKeys[ind]
        nextKey = sortedKeys[ind+1] if ind+1 < len(sortedKeys) else len(tokenizedSentence)
        sentenceChunks[key] = generateSentenceChunks(tokenizedSentence, key, nextKey, indexToWordDict[key])

    if(sortedKeys[0] != 0):
        newList = []
        for thislist in sentenceChunks[sortedKeys[0]]:
            newList.append(tokenizedSentence[0:sortedKeys[0]] + thislist)
        sentenceChunks[sortedKeys[0]] = newList;
        
    finalOptions = combineSentenceChunks(tokenizedSentence, sentenceChunks)
    
    finalSentences = []
    for val in finalOptions:
        sentence = ' '.join(val)
        sentiment = senty.polarity_scores(sentence)['compound']
        finalSentences.append(SentenceWithSentiment(sentence,sentiment))
    
    sentenceObj.addFinalSentences(finalSentences)   
    return sentenceObj



        

In [124]:
def getAlternativeWords(sentenceObj):
    mainSentence = sentenceObj.ogSentence;
    mainSentiment = sentenceObj.ogSentiment;

    sentenceTokens = cleanAndTokenizeText(mainSentence)

    for ind, word in enumerate(sentenceTokens):
        alternativeSentenceWithHTML = []
        copyOfTokens = sentenceTokens[:]
        replacements = []
        
        score = senty.polarity_scores(word)['compound']
        if(score != 0.0):
            replacements = sentenceObj.checkIfWordExists(word)
            if(replacements != []):
                print("FOUND REPEATED WORD {0}".format(word))
                sentenceObj.addAlternativesByIndex(ind, replacements)
                continue;
            tempReplacements = returnReplacementsForWord(word) # get embedding based relations
            if(VERBOSE_PRINTING): print("Some early replacements for word [{0}] are {1}".format(word, tempReplacements[0:8]))
            if(tempReplacements == []):
                print("No replacements found at all for word {0}".format(word))
                continue
            replacements = posApprovedReplacements(tempReplacements[:], copyOfTokens[:], ind)
            finalReplacements = []
            if(VERBOSE_PRINTING): skippedWords = []
            for new_word in replacements:
                thisSentiment = senty.polarity_scores(new_word)['compound']
                if(thisSentiment != 0.0):
                    finalReplacements.append(new_word)
                else:
                    if(VERBOSE_PRINTING):
#                         print("Sentiment of Skipped word {} is {}".format(word, senty.polarity_scores(word)))
                        skippedWords.append(new_word)
            if(VERBOSE_PRINTING and len(skippedWords) > 0):
                print("some skipped words: {0}".format(skippedWords[:5]));
            if(finalReplacements == []):
                if(VERBOSE_PRINTING): print(" -- No POS approved words! -- for word {0}\n some non-POS:{1}".format(word, tempReplacements[:4]))
                continue
            sentenceObj.addAlternativesByIndex(ind, finalReplacements)
            sentenceObj.addWordToAlternatives(word, finalReplacements)
    return sentenceObj




In [125]:
def interactionHelper(dictOfAltObjs, sentiments, thisSent, lastWorkingIndex=0):
    if(thisSent in sentiments):
        mainIndex = sentiments.index(thisSent)
        
    else:
        mainIndex = lastWorkingIndex
    
    numLower =  mainIndex  # Mod to keep it in range 0 -> val 
    numHigher = len(sentiments) - mainIndex - 1

    print("There are totally {0} possible sentences".format(len(sentiments)))
    print("Current sentiment is at index {0}, and length of sentiments is {1}".format(mainIndex, len(sentiments)));
    print(" <--- {0} lower and {1} ---> higher ".format(numLower, numHigher))
    print("Would you like to see a lower sentiment or higher sentiment or ALL?")
    print("Please enter 1 for lower, 2 for higher or 0 for all")
    
    targetSentiment = thisSent
    textInput = input()
    if(textInput == ""):
        print("You entered nothing! exiting")
        return
        
    if(textInput.isdigit()):
        textInput = int(textInput)
        if(textInput == 0):
            if(VERBOSE_PRINTING): print("Printing sentiments from : {0}".format(sentiments))
            for sentiment in sentiments:
                obj = dictOfAltObjs[sentiment][0];
                displayText("{0}  : {1}".format(obj.getHTML(), sentiment))
            return
        elif(textInput == 1 and numLower > 0 and (mainIndex - 1) >= 0):
            targetSentiment = sentiments[mainIndex - 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        elif(textInput == 2 and numHigher < len(sentiments) and (mainIndex + 1) < len(sentiments)):
            targetSentiment = sentiments[mainIndex + 1]
            htmlSentence = dictOfAltObjs[targetSentiment][0].getHTML();
            displayText("{0} : {1}".format(htmlSentence, targetSentiment))
            
        else:
            print("You entered an invalid option")
            interactionHelper(dictOfAltObjs, sentiments, thisSent, mainIndex)
            return
        
    interactionHelper(dictOfAltObjs, sentiments, targetSentiment,mainIndex)



def interactionWithUser(mainSentenceObj, dictOfAltObjs,sentiments, isUser=False):
    if(isUser == False):
        return
    else:
        mainSentence = mainSentenceObj.ogSentence;
        mainSent = mainSentenceObj.ogSentiment;
        if(VERBOSE_PRINTING):
            print("The following are the sentiment values for the input")
            for ind,sent in enumerate(sentiments):
                print("{0} - {1}".format(ind+1,sent))
        
        interactionHelper(dictOfAltObjs, sentiments, mainSent) 

In [126]:
def extractTwitterDataset():
    df_tweets = pd.read_csv( pathToDatasets + 'cleanedTweets.csv', nrows=NUM_OF_TWEETS, skiprows =TWEET_START)
    tweets = df_tweets.values
    return tweets;

def createDictionaryOfSentStrings(sentencesWithSentiment):
    dict_sentimentToStringObjects = {}
    for obj in sentencesWithSentiment:
        sent = obj.getSentiment()
        if(dict_sentimentToStringObjects.get(sent) == None):
            dict_sentimentToStringObjects[sent] = [obj]
        else:
            (dict_sentimentToStringObjects[sent]).append(obj)

    return dict_sentimentToStringObjects;

def runThroughTweets():
    tweets = extractTwitterDataset()
    
    counterOfTweets = 0;
    for counter,tweet in enumerate(tweets):
        counterOfTweets +=1;
        tweet = tweet[0]
        tweetTokens = cleanAndTokenizeText(tweet)
        mainSentiment = senty.polarity_scores(tweet)['compound']
        if(mainSentiment == 0):
            continue
        print("\n\n Tweet: {0}:{1}\n".format(tweet,mainSentiment))   
        sentenceObj = Sentence(tweet, mainSentiment)
        sentenceObj = getAlternativeWords(sentenceObj)
        replacementDictionary = sentenceObj.getDictOfIndexWords();
        if(len(replacementDictionary) <= 0):        
            print(" -- No new Strings generated ---\n\n")
            continue

        keysToChange = replacementDictionary.keys();
        for key in keysToChange:
            if(VERBOSE_PRINTING):print("{0}'th word's ({2}) options: {1}".format(key,replacementDictionary[key],tweetTokens[key]))

        sentenceObj = returnCombinationsOfStrings(sentenceObj)
        sentenceObj = removeNegationsFromSentence(sentenceObj)
        sentencesWithSentiment, sentenceObj = printStrings(sentenceObj)
       
        dict_sentimentToStringObjects = createDictionaryOfSentStrings(sentencesWithSentiment)
        
        sentiments = sorted(dict_sentimentToStringObjects.keys())
        
    return counterOfTweets;
        
numOfTweets = runThroughTweets()
print("num of tweets done : {0}".format(numOfTweets))



 Tweet: mraow i feel like dancing but first art school wants to rape me some more. :-0.7783

Some antonyms for word like are ['unlike', 'unalike', 'dislike']
Some synonyms for word like are ['similar', 'alike', 'comparable']
Some early replacements for word [like] are ['that', 'look', 'just', "n't", 'it', 'think', 'know', 'unlike']
some skipped words: ['that', 'unlike']
No antonyms found for word rape
Some synonyms for word rape are ['despoil', 'violate', 'spoil']
Some early replacements for word [rape] are ['murder', 'abuse', 'victim', 'assault', 'raped', 'raping', 'harassment', 'despoil']
some skipped words: ['despoil', 'spoil', 'dishonour', 'colza', 'ravishment']
3'th word's (like) options: {'like'}
11'th word's (rape) options: {'violate', 'abuse', 'harassment', 'rape', 'assault', 'murder', 'violation', 'outrage', 'victim'}
number of negations removed: 0
--- More options (total: 20) possible, but not printed ---




 Tweet:  ah! i bet it does i have been like craving to play softball or basketball !!! but i have noo time :0.56

Some antonyms for word like are ['unlike', 'unalike', 'dislike']
Some synonyms for word like are ['similar', 'alike', 'comparable']
Some early replacements for word [like] are ['that', 'look', 'just', "n't", 'it', 'think', 'know', 'unlike']
some skipped words: ['unlike']
Some antonyms for word play are ['tightness']
Some synonyms for word play are ['gambol', 'roleplay', 'bid']
Some early replacements for word [play] are ['playing', 'game', 'played', 'plays', 'games', 'let', 'beat', 'tightness']
some skipped words: ['let', 'beat', 'gambol', 'roleplay', 'bid']
9'th word's (like) options: {'like'}
12'th word's (play) options: {'fun', 'play'}
number of negations removed: 0




 Tweet: is terrified she accidentally deleted a reference in her management assignment and hopes she doesnt get in trouble when she gets back :-0.3274

No antonyms found for word terrified
Some synonyms for word terrified are ['panicky', 'panic-struck', 'terrified']
Some early replacements for word [terrified] are ['scared', 'afraid', 'shitless', 'frightened', 'terrifying', 'freaked', 'embarrassed', 'panicky']
Some antonyms for word accidentally are ['intentionally', 'deliberately']
Some synonyms for word accidentally are ['incidentally', 'unexpectedly', 'unintentionally']
Some early replacements for word [accidentally] are ['accidently', 'randomly', 'tried', 'purposely', 'casually', 'forgot', 'caught', 'intentionally']
some skipped words: ['accidently', 'randomly', 'purposely', 'intentionally', 'deliberately']
Some antonyms for word hopes are ['despair']
Some synonyms for word hopes are ['go_for', 'hop-skip', 'promise']
Some early replacements for word [hopes] are ['fears', 'hoping'



 Tweet: amazon s3 plugin not worked in my website . it need php 5 . but my hosting provider has only php 4.4  . very sad.:-0.6416

No antonyms found for word amazon
Some synonyms for word amazon are ['virago', 'amazon_river', 'amazon']
Some early replacements for word [amazon] are ['kindle', 'ebay', 'ebook', 'store', 'blu-ray', 'deals', 'gc', 'virago']
some skipped words: ['kindle', 'ebook']
Some antonyms for word sad are ['glad']
Some synonyms for word sad are ['distressing', 'sorry', 'lamentable']
Some early replacements for word [sad] are ['depressing', 'really', 'depressed', 'sick', 'feel', 'cry', 'upset', 'glad']
some skipped words: ['deplorable']
0'th word's (amazon) options: {'amazon'}
24'th word's (sad) options: {'distressing', 'depressing', 'sorry', 'sick', 'lamentable', 'upset', 'depressed', 'sad', 'pitiful', 'glad'}
number of negations removed: 22
--- More options (total: 44) possible, but not printed ---




 Tweet: argh! i was suuuper sleepy an hour ago now i am wide awake.  hope i do not stay up all night. :0.4926

Some antonyms for word hope are ['despair']
Some synonyms for word hope are ['go_for', 'promise', 'desire']
Some early replacements for word [hope] are ['hopefully', 'well', 'hoping', 'everyone', "'ll", 'sure', 'see', 'despair']
15'th word's (hope) options: {'hope'}
number of negations removed: 2


...



 Tweet:  the weather in canada is freezing :-0.1027

Some antonyms for word freezing are ['unblock', 'boil', 'unfreeze']
Some synonyms for word freezing are ['freezing', 'immobilize', 'freeze_out']
Some early replacements for word [freezing] are ['cold', 'weather', 'degrees', 'snowing', 'outside', 'snow', 'raining', 'unblock']
some skipped words: ['snowing', 'raining']
5'th word's (freezing) options: {'freezing'}
number of negations removed: 0




 Tweet: twiggassssssss i been out of range all day  i am back now and hopefully for good.:0.6808

Some antonyms for word hopefully are ['hopelessly']
Some synonyms for word hopefully are ['hopefully']
Some early replacements for word [hopefully] are ['hope', 'hoping', 'soon', 'maybe', 'well', 'tomorrow', 'ill', 'hopelessly']
some skipped words: ['soon', 'maybe']
Some antonyms for word good are ['bad', 'evil', 'evilness']
Some synonyms for word good are ['in_effect', 'salutary', 'trade_good']
Some early replacements for word [good] are ['great', 'well', 'nice', 'better', 'night', 'morning', 'but', 'bad']
some skipped words: ['adept', 'skillful', 'undecomposed', 'skilful', 'upright']
13'th word's (hopefully) options: {'well', 'hopelessly', 'hopefully', 'ill'}
15'th word's (good) options: {'serious', 'bad', 'honorable', 'nice', 'good', 'beneficial', 'honest', 'great', 'safe', 'secure', 'effective', 'respectable', 'ill'}
number of negations removed: 0
--- More options (total: 70) possibl



 Tweet: anyone who reads this pray for my grandma. she is in pain :-0.1717

No antonyms found for word pray
Some synonyms for word pray are ['beg', 'pray', 'implore']
Some early replacements for word [pray] are ['praying', 'prayers', 'god', 'bless', 'hope', 'prayer', 'prayfor', 'beg']
some skipped words: ['prayer', 'prayfor', 'beg', 'implore']
Some antonyms for word pain are ['pleasure']
Some synonyms for word pain are ['pain_sensation', 'annoyance', 'painfulness']
Some early replacements for word [pain] are ['hurts', 'pains', 'hurt', 'stress', 'sadness', 'suffering', 'painful', 'pleasure']
some skipped words: ['infliction', 'ail', 'nuisance']
4'th word's (pray) options: {'pray', 'god', 'bless', 'hope'}
12'th word's (pain) options: {'suffering', 'annoyance', 'pleasure', 'painfulness', 'trouble', 'hurt', 'anguish', 'sadness', 'stress', 'botheration', 'pain', 'bother'}
number of negations removed: 0
--- More options (total: 65) possible, but not printed ---




 Tweet: 3 woke up and was having an accident  quotit is pushing it is pushing!quot he was crying because he could not stop from wetting his pants.  :-0.6812

No antonyms found for word accident
Some synonyms for word accident are ['chance_event', 'fortuity', 'accident']
Some early replacements for word [accident] are ['crash', 'collision', 'vehicle', 'incident', 'highway', 'near', 'road', 'chance_event']
some skipped words: ['vehicle', 'incident', 'highway', 'road', 'stroke']
Some antonyms for word crying are ['laugh']
Some synonyms for word crying are ['call', 'cry', 'vociferation']
Some early replacements for word [crying] are ['laughing', 'cry', 'sobbing', 'dying', 'screaming', 'literally', 'omfg', 'laugh']
some skipped words: ['dying', 'glaring']
Some antonyms for word stop are ['begin', 'start', 'continuant_consonant']
Some synonyms for word stop are ['stopover', 'contain', 'stoppage']
Some early replacements for word [stop] are ['trying', "n't", 'keep', 'why', 'quit', 'just', '

num of tweets done : 100


## Testing Code


In [127]:
specificWord = "good"
def testOneWord(word=""):
    if(word==""):
        return
    print(word)

In [128]:
specificString = ""
def specificString(textString=""):
    if(textString == "" or textString == None):
        return
    textString = dataClean(textString)
    mainSentiment = senty.polarity_scores(textString)['compound']
    if(mainSentiment == 0):
        print("{} \n No sentiment found in sentence".format(textString));
        return;
    print("\n {0}:{1}\n".format(textString,mainSentiment))   
    sentenceObj = Sentence(textString, mainSentiment)
    sentenceObj = getAlternativeWords(sentenceObj)
    replacementDictionary = sentenceObj.getDictOfIndexWords();
    if(len(replacementDictionary) <= 0):        
        print(" -- No new Strings generated ---\n\n")
        return
    
    keysToChange = replacementDictionary.keys();
    if(VERBOSE_PRINTING):
        for key in keysToChange:
            print("{0}'th word's options: {1}".format(key,replacementDictionary[key]))
        
    
    sentenceObj = returnCombinationsOfStrings(sentenceObj)
    sentenceObj = removeNegationsFromSentence(sentenceObj)
    allPossibleSentences, sentenceObj = printStrings(sentenceObj)
    
    
    dict_sentimentToStringObjects = createDictionaryOfSentStrings(allPossibleSentences)
        
    sentiments = sorted(dict_sentimentToStringObjects.keys())

print("Enter a string to use")


inputText = input()
if(inputText == ""):
    return
elif(inputText == "t"):
    specificString("I really love hot chocolate, but no")
else:
    specificString(inputText)
# I really love hot chocolate, but I'm not good with hot coffee 

Enter a string to use




this is making me feel bad:-0.5423
this is making me feel a bit bad:-0.5423

this is making me feel very bad:-0.5849
this is making me feel quite bad:-0.5849
this is making me feel incredibly bad:-0.5849

this is making me feel kinda bad:-0.4951
this is making me feel slightly bad:-0.4951
this is making me feel barely bad:-0.4951

this is making me feel not bad:0.431
